In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pygad
global num_clusters,feature_vector_length,df

df=pd.read_csv(r'C:\Users\KARAN\Desktop\BBS Sir Proj\wine-clustering.csv',encoding='unicode_escape')
df

,Alcohol,Malic_Acid,Ash,Ash_Alcanity,Magnesium,Total_Phenols,Flavanoids,Nonflavanoid_Phenols,Proanthocyanins,Color_Intensity,Hue,OD280,Proline
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840


In [10]:
def euclidean_distance(X, Y):
    return np.sqrt(np.sum(np.power(X - Y, 2), axis=1))

def cluster_data(solution):
    cluster_centers = []
    all_clusters_dists = []
    clusters = []
    clusters_sum_dist = []

    for clust_idx in range(num_clusters):
        cluster_centers.append(solution[feature_vector_length * clust_idx:feature_vector_length * (clust_idx + 1)])
        cluster_center_dists = euclidean_distance(df.values, cluster_centers[clust_idx])
        all_clusters_dists.append(np.array(cluster_center_dists))

    cluster_centers = np.array(cluster_centers)
    all_clusters_dists = np.array(all_clusters_dists)

    cluster_indices = np.argmin(all_clusters_dists, axis=0)
    for clust_idx in range(num_clusters):
        clusters.append(np.where(cluster_indices == clust_idx)[0])
        if len(clusters[clust_idx]) == 0:
            clusters_sum_dist.append(0)
        else:
            clusters_sum_dist.append(np.sum(all_clusters_dists[clust_idx, clusters[clust_idx]]))

    clusters_sum_dist = np.array(clusters_sum_dist)

    return cluster_centers, all_clusters_dists, cluster_indices, clusters, clusters_sum_dist

def fitness_func(ga_instance,solution, solution_idx):
    _, _, _, _, clusters_sum_dist = cluster_data(solution)

    fitness = 1.0 / (np.sum(clusters_sum_dist) + 0.00000001)

    return fitness

import pygad
num_clusters = 3
feature_vector_length = df.shape[1]
num_genes = num_clusters * feature_vector_length

ga_instance = pygad.GA(num_generations=10000,
                    sol_per_pop=8,
                    init_range_low=-2,
                    num_parents_mating=4,
                    keep_parents=1,
                    parent_selection_type="sss",
                    crossover_type="uniform",
                    crossover_probability=0.5,
                    mutation_type="adaptive",
                    mutation_probability=[0.25,0.5],
                    init_range_high=5,
                    num_genes=num_genes,
                    fitness_func=fitness_func,
                    suppress_warnings=True)

ga_instance.run()

best_solution, best_solution_fitness, best_solution_idx = ga_instance.best_solution()
print("Best solution is {bs}".format(bs=best_solution))
print("Fitness of the best solution is {bsf}".format(bsf=best_solution_fitness))
print("Best solution found after {gen} generations".format(gen=ga_instance.best_solution_generation))
cluster_centers, all_clusters_dists, cluster_indices, clusters, clusters_sum_dist = cluster_data(best_solution)
X=df
labels=cluster_indices
from sklearn.metrics import silhouette_score
silhouette_avg = silhouette_score(X, labels)
print("Silhouette Score:", silhouette_avg)
from sklearn.metrics import davies_bouldin_score
db_index = davies_bouldin_score(X, labels)
print("Davies-Bouldin Index:", db_index)
from sklearn.metrics import calinski_harabasz_score
ch_index = calinski_harabasz_score(X, labels)
print("Calinski-Harabasz Index:", ch_index)


Best solution is [ 2.38407580e+01 -1.44054234e+01 -4.74485943e+00  1.02026821e+01
 -1.46816734e+01  2.77787113e+01 -9.30005615e-01  7.32591644e-01
  2.11535913e+01  9.24316730e-02  2.00464405e+01 -1.77568609e+00
  2.18593821e+01  1.25883792e+01  2.93193598e+00  2.05421469e+00
  2.10146664e+01  9.47903791e+01  1.78742842e+00  1.03562197e+00
  4.85431809e-01  1.33670333e+00  4.74175926e+00  8.03063051e-01
  2.53244415e+00  5.48452929e+02  1.32281524e+01  2.09589834e+00
  2.81327875e+00  1.77869893e+01  1.06977737e+02  2.75792112e+00
  2.69119088e+00  2.01628164e-01  1.17465675e+00  5.28307888e+00
  1.01705716e+00  2.87873562e+00  1.07942082e+03]
Fitness of the best solution is 4.27571192705128e-05
Best solution found after 9583 generations
Silhouette Score: 0.6493647595706149
Davies-Bouldin Index: 0.49620599247150626
Calinski-Harabasz Index: 498.886164329481


In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pygad
global num_clusters,feature_vector_length,df

df=pd.read_csv(r'C:\Users\KARAN\Desktop\BBS Sir Proj\iris-clustering.csv',encoding='unicode_escape')
df

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
142,6.7,3.0,5.2,2.3
143,6.3,2.5,5.0,1.9
144,6.5,3.0,5.2,2.0
145,6.2,3.4,5.4,2.3


In [12]:
def euclidean_distance(X, Y):
    return np.sqrt(np.sum(np.power(X - Y, 2), axis=1))

def cluster_data(solution):
    cluster_centers = []
    all_clusters_dists = []
    clusters = []
    clusters_sum_dist = []

    for clust_idx in range(num_clusters):
        cluster_centers.append(solution[feature_vector_length * clust_idx:feature_vector_length * (clust_idx + 1)])
        cluster_center_dists = euclidean_distance(df.values, cluster_centers[clust_idx])
        all_clusters_dists.append(np.array(cluster_center_dists))

    cluster_centers = np.array(cluster_centers)
    all_clusters_dists = np.array(all_clusters_dists)

    cluster_indices = np.argmin(all_clusters_dists, axis=0)
    for clust_idx in range(num_clusters):
        clusters.append(np.where(cluster_indices == clust_idx)[0])
        if len(clusters[clust_idx]) == 0:
            clusters_sum_dist.append(0)
        else:
            clusters_sum_dist.append(np.sum(all_clusters_dists[clust_idx, clusters[clust_idx]]))

    clusters_sum_dist = np.array(clusters_sum_dist)

    return cluster_centers, all_clusters_dists, cluster_indices, clusters, clusters_sum_dist

def fitness_func(ga_instance,solution, solution_idx):
    _, _, _, _, clusters_sum_dist = cluster_data(solution)

    fitness = 1.0 / (np.sum(clusters_sum_dist) + 0.00000001)

    return fitness

import pygad
num_clusters = 3
feature_vector_length = df.shape[1]
num_genes = num_clusters * feature_vector_length

ga_instance = pygad.GA(num_generations=10000,
                    sol_per_pop=8,
                    init_range_low=-2,
                    num_parents_mating=4,
                    keep_parents=1,
                    parent_selection_type="sss",
                    crossover_type="uniform",
                    crossover_probability=0.5,
                    mutation_type="adaptive",
                    mutation_probability=[0.25,0.5],
                    init_range_high=5,
                    num_genes=num_genes,
                    fitness_func=fitness_func,
                    suppress_warnings=True)

ga_instance.run()

best_solution, best_solution_fitness, best_solution_idx = ga_instance.best_solution()
print("Best solution is {bs}".format(bs=best_solution))
print("Fitness of the best solution is {bsf}".format(bsf=best_solution_fitness))
print("Best solution found after {gen} generations".format(gen=ga_instance.best_solution_generation))
cluster_centers, all_clusters_dists, cluster_indices, clusters, clusters_sum_dist = cluster_data(best_solution)
X=df
labels=cluster_indices
from sklearn.metrics import silhouette_score
silhouette_avg = silhouette_score(X, labels)
print("Silhouette Score:", silhouette_avg)
from sklearn.metrics import davies_bouldin_score
db_index = davies_bouldin_score(X, labels)
print("Davies-Bouldin Index:", db_index)
from sklearn.metrics import calinski_harabasz_score
ch_index = calinski_harabasz_score(X, labels)
print("Calinski-Harabasz Index:", ch_index)


Best solution is [ 6.20423246  2.89736927  4.90418559  1.70890389  5.01798658  3.42633035
  1.46981823  0.2390132   2.7290041   1.54287657 -5.57908485 -4.2099779 ]
Fitness of the best solution is 0.007899723171102989
Best solution found after 9002 generations
Silhouette Score: 0.6794971398933359
Davies-Bouldin Index: 0.40052718060567716
Calinski-Harabasz Index: 487.27483423419113


In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pygad
global num_clusters,feature_vector_length,df

df=pd.read_csv(r'C:\Users\KARAN\Desktop\BBS Sir Proj\haberman-clustering.csv',encoding='unicode_escape')
df

,Age,Op_year,axil_nodes
0,30,62,3
1,30,65,0
2,31,59,2
3,31,65,4
4,33,58,10
...,...,...,...
277,75,62,1
278,76,67,0
279,77,65,3
280,78,65,1


In [15]:
def euclidean_distance(X, Y):
    return np.sqrt(np.sum(np.power(X - Y, 2), axis=1))

def cluster_data(solution):
    cluster_centers = []
    all_clusters_dists = []
    clusters = []
    clusters_sum_dist = []

    for clust_idx in range(num_clusters):
        cluster_centers.append(solution[feature_vector_length * clust_idx:feature_vector_length * (clust_idx + 1)])
        cluster_center_dists = euclidean_distance(df.values, cluster_centers[clust_idx])
        all_clusters_dists.append(np.array(cluster_center_dists))

    cluster_centers = np.array(cluster_centers)
    all_clusters_dists = np.array(all_clusters_dists)

    cluster_indices = np.argmin(all_clusters_dists, axis=0)
    for clust_idx in range(num_clusters):
        clusters.append(np.where(cluster_indices == clust_idx)[0])
        if len(clusters[clust_idx]) == 0:
            clusters_sum_dist.append(0)
        else:
            clusters_sum_dist.append(np.sum(all_clusters_dists[clust_idx, clusters[clust_idx]]))

    clusters_sum_dist = np.array(clusters_sum_dist)

    return cluster_centers, all_clusters_dists, cluster_indices, clusters, clusters_sum_dist

def fitness_func(ga_instance,solution, solution_idx):
    _, _, _, _, clusters_sum_dist = cluster_data(solution)

    fitness = 1.0 / (np.sum(clusters_sum_dist) + 0.00000001)

    return fitness

import pygad
num_clusters = 2
feature_vector_length = df.shape[1]
num_genes = num_clusters * feature_vector_length

ga_instance = pygad.GA(num_generations=10000,
                    sol_per_pop=8,
                    init_range_low=-2,
                    num_parents_mating=4,
                    keep_parents=1,
                    parent_selection_type="sss",
                    crossover_type="uniform",
                    crossover_probability=0.5,
                    mutation_type="adaptive",
                    mutation_probability=[0.25,0.5],
                    init_range_high=5,
                    num_genes=num_genes,
                    fitness_func=fitness_func,
                    suppress_warnings=True)

ga_instance.run()

best_solution, best_solution_fitness, best_solution_idx = ga_instance.best_solution()
print("Best solution is {bs}".format(bs=best_solution))
print("Fitness of the best solution is {bsf}".format(bsf=best_solution_fitness))
print("Best solution found after {gen} generations".format(gen=ga_instance.best_solution_generation))
cluster_centers, all_clusters_dists, cluster_indices, clusters, clusters_sum_dist = cluster_data(best_solution)
X=df
labels=cluster_indices
from sklearn.metrics import silhouette_score
silhouette_avg = silhouette_score(X, labels)
print("Silhouette Score:", silhouette_avg)
from sklearn.metrics import davies_bouldin_score
db_index = davies_bouldin_score(X, labels)
print("Davies-Bouldin Index:", db_index)
from sklearn.metrics import calinski_harabasz_score
ch_index = calinski_harabasz_score(X, labels)
print("Calinski-Harabasz Index:", ch_index)


Best solution is [-20.57336378  -0.63620811 -28.67507102  51.97575336  62.78559266
   2.90533367]
Fitness of the best solution is 0.0003046539567980344
Best solution found after 8960 generations


ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pygad
global num_clusters,feature_vector_length,df

df=pd.read_csv(r'C:\Users\KARAN\Desktop\BBS Sir Proj\glass-clustering.csv',encoding='unicode_escape')
df

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.0
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.00,0.0
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.00,0.0
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.00,0.0
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.00,0.0
...,...,...,...,...,...,...,...,...,...
208,1.51623,14.14,0.00,2.88,72.61,0.08,9.18,1.06,0.0
209,1.51685,14.92,0.00,1.99,73.06,0.00,8.40,1.59,0.0
210,1.52065,14.36,0.00,2.02,73.42,0.00,8.44,1.64,0.0
211,1.51651,14.38,0.00,1.94,73.61,0.00,8.48,1.57,0.0


In [19]:
def euclidean_distance(X, Y):
    return np.sqrt(np.sum(np.power(X - Y, 2), axis=1))

def cluster_data(solution):
    cluster_centers = []
    all_clusters_dists = []
    clusters = []
    clusters_sum_dist = []

    for clust_idx in range(num_clusters):
        cluster_centers.append(solution[feature_vector_length * clust_idx:feature_vector_length * (clust_idx + 1)])
        cluster_center_dists = euclidean_distance(df.values, cluster_centers[clust_idx])
        all_clusters_dists.append(np.array(cluster_center_dists))

    cluster_centers = np.array(cluster_centers)
    all_clusters_dists = np.array(all_clusters_dists)

    cluster_indices = np.argmin(all_clusters_dists, axis=0)
    for clust_idx in range(num_clusters):
        clusters.append(np.where(cluster_indices == clust_idx)[0])
        if len(clusters[clust_idx]) == 0:
            clusters_sum_dist.append(0)
        else:
            clusters_sum_dist.append(np.sum(all_clusters_dists[clust_idx, clusters[clust_idx]]))

    clusters_sum_dist = np.array(clusters_sum_dist)

    return cluster_centers, all_clusters_dists, cluster_indices, clusters, clusters_sum_dist

def fitness_func(ga_instance,solution, solution_idx):
    _, _, _, _, clusters_sum_dist = cluster_data(solution)

    fitness = 1.0 / (np.sum(clusters_sum_dist) + 0.00000001)

    return fitness

import pygad
num_clusters = 6
feature_vector_length = df.shape[1]
num_genes = num_clusters * feature_vector_length

ga_instance = pygad.GA(num_generations=10000,
                    sol_per_pop=8,
                    init_range_low=-2,
                    num_parents_mating=4,
                    keep_parents=1,
                    parent_selection_type="sss",
                    crossover_type="uniform",
                    crossover_probability=0.5,
                    mutation_type="adaptive",
                    mutation_probability=[0.25,0.5],
                    init_range_high=5,
                    num_genes=num_genes,
                    fitness_func=fitness_func,
                    suppress_warnings=True)

ga_instance.run()

best_solution, best_solution_fitness, best_solution_idx = ga_instance.best_solution()
print("Best solution is {bs}".format(bs=best_solution))
print("Fitness of the best solution is {bsf}".format(bsf=best_solution_fitness))
print("Best solution found after {gen} generations".format(gen=ga_instance.best_solution_generation))
cluster_centers, all_clusters_dists, cluster_indices, clusters, clusters_sum_dist = cluster_data(best_solution)
X=df
labels=cluster_indices
from sklearn.metrics import silhouette_score
silhouette_avg = silhouette_score(X, labels)
print("Silhouette Score:", silhouette_avg)
from sklearn.metrics import davies_bouldin_score
db_index = davies_bouldin_score(X, labels)
print("Davies-Bouldin Index:", db_index)
from sklearn.metrics import calinski_harabasz_score
ch_index = calinski_harabasz_score(X, labels)
print("Calinski-Harabasz Index:", ch_index)


Best solution is [-3.91545997e-02 -5.23883549e-01 -5.12960705e+00  1.20628227e+01
  2.77430558e+00 -1.07106522e+00 -7.38334646e+00 -8.89906924e+00
  1.22143653e+00 -6.18800480e+00  1.28317729e+00  1.77959078e+00
 -1.30411383e+01 -3.98552907e+00 -8.45935590e+00 -1.22006103e+00
  9.88324829e+00 -2.33097034e+00 -1.68534130e-02  2.83856050e+00
  5.01082239e+00  6.31680740e+00 -7.82265370e+00  1.11661511e+01
  1.21889789e+01  4.71891751e+00 -1.41165489e+00  4.69682877e+00
  3.00535614e+00 -1.34321924e+00 -6.66072191e+00  1.59964361e+00
 -1.53019051e+00 -4.53582232e+00 -1.64670332e-01  4.16624544e+00
  5.41915685e+00 -6.66318960e-01  4.85861490e+00 -5.72344855e+00
  1.48420573e+01  3.44093875e+00 -2.28215588e+00  5.42295700e+00
  9.61320552e+00  1.50294744e+00  1.32530129e+01  3.21038771e+00
  1.34449969e+00  7.26858055e+01  5.21613435e-01  8.59833231e+00
  4.18546582e-02  2.28637468e-02]
Fitness of the best solution is 0.0024406849316300347
Best solution found after 7897 generations


ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pygad
global num_clusters,feature_vector_length,df

df=pd.read_csv(r'C:\Users\KARAN\Desktop\BBS Sir Proj\breast-cancer-clustering.csv',encoding='unicode_escape')
df

In [ ]:
def euclidean_distance(X, Y):
    return np.sqrt(np.sum(np.power(X - Y, 2), axis=1))

def cluster_data(solution):
    cluster_centers = []
    all_clusters_dists = []
    clusters = []
    clusters_sum_dist = []

    for clust_idx in range(num_clusters):
        cluster_centers.append(solution[feature_vector_length * clust_idx:feature_vector_length * (clust_idx + 1)])
        cluster_center_dists = euclidean_distance(df.values, cluster_centers[clust_idx])
        all_clusters_dists.append(np.array(cluster_center_dists))

    cluster_centers = np.array(cluster_centers)
    all_clusters_dists = np.array(all_clusters_dists)

    cluster_indices = np.argmin(all_clusters_dists, axis=0)
    for clust_idx in range(num_clusters):
        clusters.append(np.where(cluster_indices == clust_idx)[0])
        if len(clusters[clust_idx]) == 0:
            clusters_sum_dist.append(0)
        else:
            clusters_sum_dist.append(np.sum(all_clusters_dists[clust_idx, clusters[clust_idx]]))

    clusters_sum_dist = np.array(clusters_sum_dist)

    return cluster_centers, all_clusters_dists, cluster_indices, clusters, clusters_sum_dist

def fitness_func(ga_instance,solution, solution_idx):
    _, _, _, _, clusters_sum_dist = cluster_data(solution)

    fitness = 1.0 / (np.sum(clusters_sum_dist) + 0.00000001)

    return fitness

import pygad
num_clusters = 2
feature_vector_length = df.shape[1]
num_genes = num_clusters * feature_vector_length

ga_instance = pygad.GA(num_generations=10000,
                    sol_per_pop=8,
                    init_range_low=-2,
                    num_parents_mating=4,
                    keep_parents=1,
                    parent_selection_type="sss",
                    crossover_type="uniform",
                    crossover_probability=0.5,
                    mutation_type="adaptive",
                    mutation_probability=[0.25,0.5],
                    init_range_high=5,
                    num_genes=num_genes,
                    fitness_func=fitness_func,
                    suppress_warnings=True)

ga_instance.run()

best_solution, best_solution_fitness, best_solution_idx = ga_instance.best_solution()
print("Best solution is {bs}".format(bs=best_solution))
print("Fitness of the best solution is {bsf}".format(bsf=best_solution_fitness))
print("Best solution found after {gen} generations".format(gen=ga_instance.best_solution_generation))
cluster_centers, all_clusters_dists, cluster_indices, clusters, clusters_sum_dist = cluster_data(best_solution)
X=df
labels=cluster_indices
from sklearn.metrics import silhouette_score
silhouette_avg = silhouette_score(X, labels)
print("Silhouette Score:", silhouette_avg)
from sklearn.metrics import davies_bouldin_score
db_index = davies_bouldin_score(X, labels)
print("Davies-Bouldin Index:", db_index)
from sklearn.metrics import calinski_harabasz_score
ch_index = calinski_harabasz_score(X, labels)
print("Calinski-Harabasz Index:", ch_index)
